In [8]:
import os
import sys
import pandas as pd
import datetime
import numpy as np
import time
import re
from tqdm import tqdm
import timeit
tqdm.pandas()
import omap


In [38]:
from datetime import datetime

ts = int("1573750800")

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

ts = int("1603729564")

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

ts = int("1519603200")

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))


2019-11-14 17:00:00
2020-10-26 16:26:04
2018-02-26 00:00:00


In [96]:
# input_folder_path =  './EVR_T23_Car46_261020/'
# #input_folder_path = './Train 23 14NOV2019 Work Folder/raw logs'
# start_time = '2020/10/26 15:50:00' 
# end_time = '2020/10/26 16:50:00'
# #start_time = '2020/10/26 04:36:45'
# #end_time = '2020/10/26 04:37:09'
# sample_output_filename = ['./EVR_T23_Car46_261020/preprocessing_output/EVR_20201026_1550_to_20201026_1650.csv']

# input_folder_path = './Train 23 14NOV2019 Work Folder/raw logs'
# #input_folder_path = './Train 23 14NOV2019 Work Folder/raw logs'
# start_time = '2019/11/14 17:00:00' 
# end_time = '2019/11/14 18:00:00'
# sample_output_filename = ['./Train 23 14NOV2019 Work Folder/preprocessing_output/EVR_20191114_1700_to_20191114_1800.csv']

# input_folder_path = './Train 23 14NOV2019 Work Folder/raw logs'
# #input_folder_path = './Train 23 14NOV2019 Work Folder/raw logs'
# start_time = '2019/11/14 18:00:00' 
# end_time = '2019/11/14 19:00:00'
# sample_output_filename = ['./Train 23 14NOV2019 Work Folder/preprocessing_output/EVR_20191114_1800_to_20191114_1900.csv']

input_folder_path = './Train 23 14NOV2019 Work Folder/raw logs'
#input_folder_path = './Train 23 14NOV2019 Work Folder/raw logs'
start_time = '2019/11/14 17:00:00' 
end_time = '2019/11/14 20:00:00'
sample_output_filename = ['./Train 23 14NOV2019 Work Folder/preprocessing_output/EVR_20191114_1700_to_20191114_2000.csv']

def process_evr (input_folder_path, start_time, end_time):
    # Determine time difference between start and end time
    datetime_format = '%Y/%m/%d %H:%M:%S'
    start_time = datetime.datetime.strptime(start_time, datetime_format)
    end_time = datetime.datetime.strptime(end_time, datetime_format)
    start_time_new = start_time.strftime('%m/%d/%Y %H:%M:%S')
    end_time_new = end_time.strftime('%m/%d/%Y %H:%M:%S')
    # Check if EVR folder exist
    if not os.path.exists(os.path.join(input_folder_path, 'EVR')):
        print("EVR folder does not exist. Kindly check the folder strucutre. No EVR processing will be performed")
    else:
        print("EVR folder exist!")
        # Check if C2 file exist
        c2_pattern = 'EVR_Car\d+_\d+_C2.txt'
        norm_pattern = 'EVR_Car\d+_\d+.txt'
        c2_flag = False
        norm_flag = False
        for evr_file in os.listdir(os.path.join(input_folder_path, 'EVR')):
            if re.search(c2_pattern, evr_file):
                c2_filename = evr_file
                c2_flag = True
            if re.search(norm_pattern,evr_file):
                norm_filename = evr_file
                norm_flag = True

        if not (c2_flag and norm_flag):
            sys.exit("Missing EVR files! Kindly check EVR folder!")
        else:
            # Process C2 file
            df_c2 = pd.read_csv(os.path.join(input_folder_path, 'EVR', c2_filename), sep=";")
            df_c2['Date'] = pd.to_datetime(df_c2['Date'], format='%m/%d/%Y %H:%M:%S')
            df_c2_period = df_c2.loc[(df_c2['Date'] >= start_time_new) & (df_c2['Date'] <= end_time_new)]
            df_c2_clean = process_c2_dataframe(df_c2_period)
            #df_c2_clean = process_dataframe(df_c2_period, 'C2')
            #df_c2_clean = clean_dataframe_1(df_c2_period, 'C2')
            #df_c2_clean.to_csv('./result_c2.csv', index=False, header=True)
            
            # Processs Operating Data file
            df_od = pd.read_csv(os.path.join(input_folder_path, 'EVR', norm_filename), sep=";")
            df_od['Date'] = pd.to_datetime(df_od['Date'], format='%m/%d/%Y %H:%M:%S')
            df_od_period = df_od.loc[(df_od['Date'] >= start_time_new) & (df_od['Date'] <= end_time_new)]
            df_od_clean = process_od_dataframe(df_od_period, df_c2_clean)
            #df_od_clean = process_dataframe(df_od_period, 'OD')
            #df_od_clean = clean_dataframe_1(df_od_period, 'OD')
            #df_od_clean.to_csv('./result_od.csv', index=False, header=True)
            
            #df_c2_clean, df_od_clean = match_timestamp(df_c2_clean, df_od_clean)
            
            # Merge other Operating Data to C2 Dataframe based on Timestampms (ordered manner)
            df_result = pd.merge_ordered(df_c2_clean, df_od_clean, how='outer', on='Timestampms')
            
            #Remove unwanted columns
            df_result.drop(['EC2_Date', 'EC2_Timestamp', 'EOD_Date', 'EOD_Timestamp'], inplace=True, axis=1)
            #Group resulting same real timestamp together
            df_result = df_result.groupby("Timestampms").first().reset_index()
    
            #df_result.to_csv('./result_11.csv', index=False, header=True)
            
            
            #df_result = process_timestamp(df_result)
            #df_result.to_csv('./result_9.csv', index=False, header=True)
            # Merge C2 and OD files first

            #df_result.to_csv('./result_8.csv', index=False, header=True)
            # Clean Timestamp
            #df_result = preprocess_timestamp(df_result)
            
            
    return df_c2_clean, df_od_clean, df_result


df_c2_clean, df_od_clean, df_result = process_evr(input_folder_path, start_time, end_time)

#output(sample_output_filename[0], df_result)
unit_test(sample_output_filename[0], 'C2', df_c2_clean)  
unit_test(sample_output_filename[0], 'OD', df_od_clean)  
unit_test_all(sample_output_filename[0], df_result)

EVR folder exist!
1573750801000
--- Handling Duplicates, Split duplicate evenly---
13952 2019-11-14 17:00:01 1573750801000 13952
13953 2019-11-14 17:00:01 1573750801333 13953
13954 2019-11-14 17:00:01 1573750801666 13954
1573750808000
--- Handling Duplicates, Split duplicate evenly---
13961 2019-11-14 17:00:08 1573750808000 13961
13962 2019-11-14 17:00:08 1573750808500 13962
1573750809000
--- Handling Duplicates, Split duplicate evenly---
13964 2019-11-14 17:00:09 1573750809000 13964
13965 2019-11-14 17:00:09 1573750809333 13965
13963 2019-11-14 17:00:09 1573750809666 13963
1573750810000
--- Handling Duplicates, Split duplicate evenly---
13966 2019-11-14 17:00:10 1573750810000 13966
13967 2019-11-14 17:00:10 1573750810500 13967
1573750811000
--- Handling Duplicates, Split duplicate evenly---
13968 2019-11-14 17:00:11 1573750811000 13968
13969 2019-11-14 17:00:11 1573750811500 13969
1573750817000
--- Handling Duplicates, Split duplicate evenly---
13975 2019-11-14 17:00:17 1573750817000 

14301 2019-11-14 17:04:24 1573751064500 14301
1573751070000
--- Handling Duplicates, Split duplicate evenly---
14307 2019-11-14 17:04:30 1573751070000 14307
14308 2019-11-14 17:04:30 1573751070500 14308
1573751071000
--- Handling Duplicates, Split duplicate evenly---
14309 2019-11-14 17:04:31 1573751071000 14309
14310 2019-11-14 17:04:31 1573751071500 14310
1573751074000
--- Handling Duplicates, Split duplicate evenly---
14315 2019-11-14 17:04:34 1573751074000 14315
14313 2019-11-14 17:04:34 1573751074333 14313
14314 2019-11-14 17:04:34 1573751074666 14314
1573751084000
--- Handling Duplicates, Split duplicate evenly---
14325 2019-11-14 17:04:44 1573751084000 14325
14326 2019-11-14 17:04:44 1573751084500 14326
1573751085000
--- Handling Duplicates, Split duplicate evenly---
14328 2019-11-14 17:04:45 1573751085000 14328
14327 2019-11-14 17:04:45 1573751085500 14327
1573751086000
--- Handling Duplicates, Split duplicate evenly---
14330 2019-11-14 17:04:46 1573751086000 14330
14329 2019-1

15275 2019-11-14 17:18:44 1573751924000 15275
15276 2019-11-14 17:18:44 1573751924500 15276
1573751925000
--- Handling Duplicates, Split duplicate evenly---
15278 2019-11-14 17:18:45 1573751925000 15278
15277 2019-11-14 17:18:45 1573751925500 15277
1573751926000
--- Handling Duplicates, Split duplicate evenly---
15279 2019-11-14 17:18:46 1573751926000 15279
15280 2019-11-14 17:18:46 1573751926333 15280
15281 2019-11-14 17:18:46 1573751926666 15281
1573751927000
--- Handling Duplicates, Split duplicate evenly---
15282 2019-11-14 17:18:47 1573751927000 15282
15283 2019-11-14 17:18:47 1573751927333 15283
15284 2019-11-14 17:18:47 1573751927666 15284
1573751928000
--- Handling Duplicates, Split duplicate evenly---
15286 2019-11-14 17:18:48 1573751928000 15286
15285 2019-11-14 17:18:48 1573751928500 15285
1573751929000
--- Handling Duplicates, Split duplicate evenly---
15287 2019-11-14 17:18:49 1573751929000 15287
15288 2019-11-14 17:18:49 1573751929333 15288
15289 2019-11-14 17:18:49 15737

15651 2019-11-14 17:23:34 1573752214333 15651
15652 2019-11-14 17:23:34 1573752214666 15652
1573752217000
--- Handling Duplicates, Split duplicate evenly---
15655 2019-11-14 17:23:37 1573752217000 15655
15656 2019-11-14 17:23:37 1573752217333 15656
15657 2019-11-14 17:23:37 1573752217666 15657
1573752218000
--- Handling Duplicates, Split duplicate evenly---
15659 2019-11-14 17:23:38 1573752218000 15659
15658 2019-11-14 17:23:38 1573752218500 15658
1573752219000
--- Handling Duplicates, Split duplicate evenly---
15660 2019-11-14 17:23:39 1573752219000 15660
15661 2019-11-14 17:23:39 1573752219500 15661
1573752220000
--- Handling Duplicates, Split duplicate evenly---
15662 2019-11-14 17:23:40 1573752220000 15662
15663 2019-11-14 17:23:40 1573752220500 15663
1573752221000
--- Handling Duplicates, Split duplicate evenly---
15664 2019-11-14 17:23:41 1573752221000 15664
15665 2019-11-14 17:23:41 1573752221500 15665
1573752222000
--- Handling Duplicates, Split duplicate evenly---
15666 2019-1

16090 2019-11-14 17:28:27 1573752507666 16090
1573752508000
--- Handling Duplicates, Split duplicate evenly---
16091 2019-11-14 17:28:28 1573752508000 16091
16092 2019-11-14 17:28:28 1573752508500 16092
1573752509000
--- Handling Duplicates, Split duplicate evenly---
16095 2019-11-14 17:28:29 1573752509000 16095
16093 2019-11-14 17:28:29 1573752509333 16093
16094 2019-11-14 17:28:29 1573752509666 16094
1573752510000
--- Handling Duplicates, Split duplicate evenly---
16096 2019-11-14 17:28:30 1573752510000 16096
16097 2019-11-14 17:28:30 1573752510250 16097
16098 2019-11-14 17:28:30 1573752510500 16098
16099 2019-11-14 17:28:30 1573752510750 16099
1573752511000
--- Handling Duplicates, Split duplicate evenly---
16100 2019-11-14 17:28:31 1573752511000 16100
16101 2019-11-14 17:28:31 1573752511333 16101
16102 2019-11-14 17:28:31 1573752511666 16102
1573752512000
--- Handling Duplicates, Split duplicate evenly---
16103 2019-11-14 17:28:32 1573752512000 16103
16104 2019-11-14 17:28:32 15737

...

1573756354000
---Handling Duplicates, Fill gap with duplicate, Forward---
21517 2019-11-14 18:32:34 1573756354000 21517
21518 2019-11-14 18:32:34 1573756355000 21518
1573756356000
--- Handling Duplicates, Split duplicate evenly---
21520 2019-11-14 18:32:36 1573756356000 21520
21519 2019-11-14 18:32:36 1573756356500 21519
1573756359000
---Handling Duplicates, Fill gap with duplicate, Forward---
21522 2019-11-14 18:32:39 1573756359000 21522
21523 2019-11-14 18:32:39 1573756360000 21523
1573756361000
--- Handling Duplicates, Split duplicate evenly---
21525 2019-11-14 18:32:41 1573756361000 21525
21524 2019-11-14 18:32:41 1573756361500 21524
1573756364000
---Handling Duplicates, Fill gap with duplicate, Forward and Backward---
21527 2019-11-14 18:32:44 1573756363000
1573756366000
---Handling Duplicates, Fill gap with duplicate, Backward---
21530 2019-11-14 18:32:46 1573756366000 21530
21529 2019-11-14 18:32:46 1573756365000 21529
1573756369000
---Handling Duplicates, Fill gap with duplicat

21749 2019-11-14 18:35:59 1573756559000 21749
21750 2019-11-14 18:35:59 1573756559500 21750
1573756577000
--- Handling Duplicates, Split duplicate evenly---
21768 2019-11-14 18:36:17 1573756577000 21768
21769 2019-11-14 18:36:17 1573756577500 21769
1573756578000
--- Handling Duplicates, Split duplicate evenly---
21770 2019-11-14 18:36:18 1573756578000 21770
21771 2019-11-14 18:36:18 1573756578333 21771
21772 2019-11-14 18:36:18 1573756578666 21772
1573756580000
--- Handling Duplicates, Split duplicate evenly---
21775 2019-11-14 18:36:20 1573756580000 21775
21774 2019-11-14 18:36:20 1573756580500 21774
1573756581000
--- Handling Duplicates, Split duplicate evenly---
21776 2019-11-14 18:36:21 1573756581000 21776
21777 2019-11-14 18:36:21 1573756581500 21777
1573756582000
--- Handling Duplicates, Split duplicate evenly---
21778 2019-11-14 18:36:22 1573756582000 21778
21779 2019-11-14 18:36:22 1573756582333 21779
21780 2019-11-14 18:36:22 1573756582666 21780
1573756583000
--- Handling Dupl

1573756824000
--- Handling Duplicates, Split duplicate evenly---
22111 2019-11-14 18:40:24 1573756824000 22111
22112 2019-11-14 18:40:24 1573756824500 22112
1573756825000
--- Handling Duplicates, Split duplicate evenly---
22113 2019-11-14 18:40:25 1573756825000 22113
22114 2019-11-14 18:40:25 1573756825500 22114
1573756826000
--- Handling Duplicates, Split duplicate evenly---
22115 2019-11-14 18:40:26 1573756826000 22115
22116 2019-11-14 18:40:26 1573756826500 22116
1573756827000
--- Handling Duplicates, Split duplicate evenly---
22117 2019-11-14 18:40:27 1573756827000 22117
22118 2019-11-14 18:40:27 1573756827500 22118
1573756828000
--- Handling Duplicates, Split duplicate evenly---
22119 2019-11-14 18:40:28 1573756828000 22119
22120 2019-11-14 18:40:28 1573756828333 22120
22121 2019-11-14 18:40:28 1573756828666 22121
1573756830000
--- Handling Duplicates, Split duplicate evenly---
22124 2019-11-14 18:40:30 1573756830000 22124
22123 2019-11-14 18:40:30 1573756830500 22123
157375683100

22349 2019-11-14 18:42:57 1573756977000 22349
22350 2019-11-14 18:42:57 1573756977500 22350
1573756984000
--- Handling Duplicates, Split duplicate evenly---
22357 2019-11-14 18:43:04 1573756984000 22357
22358 2019-11-14 18:43:04 1573756984500 22358
1573756993000
--- Handling Duplicates, Split duplicate evenly---
22367 2019-11-14 18:43:13 1573756993000 22367
22368 2019-11-14 18:43:13 1573756993333 22368
22369 2019-11-14 18:43:13 1573756993666 22369
1573756994000
--- Handling Duplicates, Split duplicate evenly---
22372 2019-11-14 18:43:14 1573756994000 22372
22371 2019-11-14 18:43:14 1573756994333 22371
22370 2019-11-14 18:43:14 1573756994666 22370
1573757001000
--- Handling Duplicates, Split duplicate evenly---
22379 2019-11-14 18:43:21 1573757001000 22379
22380 2019-11-14 18:43:21 1573757001500 22380
1573757017000
--- Handling Duplicates, Split duplicate evenly---
22396 2019-11-14 18:43:37 1573757017000 22396
22397 2019-11-14 18:43:37 1573757017500 22397
1573757022000
--- Handling Dupl

1573754297000
1 1
---Handling Duplicates, Fill gap with duplicate, Forward and Backward---
5793 2019-11-14 17:58:17 1573754296000
1573754299000
1 1
--- Handling Duplicates, Split duplicate evenly---
5796 2019-11-14 17:58:19 1573754299000 5796
5795 2019-11-14 17:58:19 1573754299500 5795
1573754301000
1 1
---Handling Duplicates, Fill gap with duplicate, Backward---
5798 2019-11-14 17:58:21 1573754301000 5798
5797 2019-11-14 17:58:21 1573754300000 5797
1573754311000
5 3
---Handling Duplicates, Fill gap with duplicate, Forward and Backward---
5807 2019-11-14 17:58:31 1573754310000
1573754314000
3 2
--- Handling Duplicates, Split duplicate evenly---
5810 2019-11-14 17:58:34 1573754314000 5810
5811 2019-11-14 17:58:34 1573754314500 5811
1573754318000
3 2
--- Handling Duplicates, Split duplicate evenly---
5815 2019-11-14 17:58:38 1573754318000 5815
5814 2019-11-14 17:58:38 1573754318500 5814
1573754321000
2 3
---Handling Duplicates, Fill gap with duplicate, Backward---
5818 2019-11-14 17:58:4

In [7]:
def clean_dataframe(df):
    # Drop columns with NaNs
    df = df.dropna(how='any',axis=1)
    # Sort df based on column 'Date'
    df = df.sort_values(by='Date',ascending=True).reset_index(drop=True)
    # Convert datetime to timestamp (Unix) in milisecond resolution
    df['Timestamp'] = df.Date.values.astype(np.int64) // 10 ** 9 *1000
    # Add column for Real_Time Timestampms
    df['Timestampms'] = df['Timestamp'] 
    
    return df

def clean_dataframe_1(df, name):
    # Drop columns with NaNs
    df = df.dropna(how='any',axis=1)
    # Sort df based on column 'Date'
    df = df.sort_values(by='Date',ascending=True).reset_index(drop=True)
    # Convert datetime to timestamp (Unix) in milisecond resolution
    df['Timestamp'] = df.Date.values.astype(np.int64) // 10 ** 9 *1000
    # Add column for Real_Time Timestampms
    df['Timestampms'] = df['Timestamp']
    
    # Add Prefix to Columns name (e.g. EC2/EOD_***) except Timestampms
    if name == 'C2':
        df.columns = df.columns.map(lambda x : 'EC2_'+x if x !='Timestampms' and x !='Timestamp' else x)
    elif name == 'OD':
        df.columns = df.columns.map(lambda x : 'EOD_'+x if x !='Timestampms' and x !='Timestamp' else x)
        
    # Shift Timestampms to first column
    df = df[ ['Timestampms'] + [ col for col in df.columns if col != 'Timestampms' ] ]
    
    return df


def match_timestamp(df_c2_clean, df_od_clean):
    df_c2_duplicate = df_c2_clean[df_c2_clean.duplicated('Timestamp',keep=False)]
    df_od_duplicate = df_od_clean[df_od_clean.duplicated('Timestamp',keep=False)]
    
    for index, unique_datetime in enumerate (df_c2_duplicate['Timestamp'].unique()):
        timestamp_df_c2 = df_c2_duplicate.loc[df_c2_duplicate['Timestamp'] == unique_datetime]
        # Difference between last and first index if it's split evenly
        difference = df_c2_clean.loc[timestamp_df_c2.index[-1]+1, 'Timestamp'] - df_c2_clean.loc[timestamp_df_c2.index[-1], 'Timestamp'] 
               
        #Check if OD timestamp + 1 second has duplicates
        timestamp_df_od = pd.DataFrame()
        timestamp_df_od = df_od_duplicate.loc[df_od_duplicate['Timestamp'] == unique_datetime+1000]
        if not timestamp_df_od.empty:
            # Split evenly for timestamp + 1 second in OD
            df_od_clean = split_equally_overall(timestamp_df_od, df_od_clean) 
            
    return df_c2_clean, df_od_clean        
            

In [87]:
def process_c2_dataframe(df):
    df =  clean_dataframe(df)
    # Find duplicates datetime in column 'Timestampms'
    df_duplicate = df[df.duplicated('Timestampms',keep=False)]

    for index, unique_datetime in enumerate (df_duplicate['Timestampms'].unique()):
        timestamp_df = df_duplicate.loc[df_duplicate['Timestampms'] == unique_datetime]
        print(unique_datetime)
        # Find difference from previous and next timestamp
        difference_previous,  difference_next = find_difference(timestamp_df, unique_datetime, df)
        
        if not(difference_previous == 0 and difference_next == 0) and ((difference_previous <= 1000 and difference_next <= 1000) or
                                                                       (difference_previous == 1000 and difference_next == 0) or
                                                                       (difference_previous == 0 and difference_next == 1000)):
    
            # Split equally e.g 1,2,2,3
            df = split_equally(timestamp_df, df)
        
        elif difference_previous <= 1000 and difference_next >=2000 and  len(timestamp_df.index) < 3:
            if df.loc[timestamp_df.index[0],'Record Number'] < df.loc[timestamp_df.index[-1],'Record Number']:
                df = split_forward(timestamp_df, df)
            else:
                # Split equally e.g 1,2,2,3
                df = split_equally(timestamp_df, df)
        
        elif difference_previous <= 1000 and difference_next >=2000 and  len(timestamp_df.index) > 2:
            if df.loc[timestamp_df.index[0],'Record Number'] < df.loc[timestamp_df.index[-1],'Record Number']:
                # split evenly then fill e.g 6.66, 7, 7, 7, 7, 9 
                df = split_evenly_fill(timestamp_df, df, difference_previous, difference_next)
            else:
                # Split equally e.g 1,2,2,3
                df = split_equally(timestamp_df, df)
            
        elif difference_previous >= 1000 and difference_next <=1000 and  len(timestamp_df.index) > 2:
            # fill then split evenly 
            df = fill_split_evenly(timestamp_df, df, difference_previous, difference_next)
                
        elif difference_previous == difference_next and difference_previous>1000 and difference_next>1000:
            if df.loc[timestamp_df.index[0],'Record Number'] < df.loc[timestamp_df.index[-1],'Record Number']:
                # fill preceding followed by behind
                df = split_forward_backward(timestamp_df, df)
            else:
                # Split equally e.g 1,2,2,3
                df = split_equally(timestamp_df, df)
                
        elif difference_previous > 1000 and difference_next <=1000 and  len(timestamp_df.index) < 3:
            if df.loc[timestamp_df.index[0],'Record Number'] < df.loc[timestamp_df.index[-1],'Record Number']:
                # Fill Backward e.g 2,4,4,5
                df = split_backward(timestamp_df, df)
            else:
                # Split equally e.g 1,2,2,3
                df = split_equally(timestamp_df, df)
            
        
    # Add Prefix to Columns name (e.g. EC2/EOD_***) except Timestampms
    df.columns = df.columns.map(lambda x : 'EC2_'+x if x !='Timestampms' else x)
        
    # Shift Timestampms to first column
    df = df[ ['Timestampms'] + [ col for col in df.columns if col != 'Timestampms' ] ]
            
    return df

def process_od_dataframe(df, df_c2):
    df =  clean_dataframe(df)
    # Find duplicates datetime in column 'Timestampms'
    df_duplicate = df[df.duplicated('Timestampms',keep=False)]
        
    for index, unique_datetime in enumerate (df_duplicate['Timestampms'].unique()):
        timestamp_df = df_duplicate.loc[df_duplicate['Timestampms'] == unique_datetime]
        
        timestamp_df_next = pd.DataFrame()
        if index+1 < len(df_duplicate['Timestampms'].unique()):
            timestamp_df_next = df_duplicate.loc[df_duplicate['Timestampms'] == df_duplicate['Timestampms'].unique()[index+1]]
        timestamp_df_previous = pd.DataFrame()
        if index > 0 :
            timestamp_df_previous = df_duplicate.loc[df_duplicate['Timestampms'] == df_duplicate['Timestampms'].unique()[index-1]]
        
        print(unique_datetime)
        
        # Check if previous timestamp in C2 has been handled (duplicates, split evenly)
        timestmap_df_c2_previous = df_c2.loc[df_c2['EC2_Timestamp'] == unique_datetime-1000]
        timestmap_df_c2_after = df_c2.loc[df_c2['EC2_Timestamp'] == unique_datetime+1000]
        
        print(len(timestmap_df_c2_previous), len(timestmap_df_c2_after))
        different_previous_c2 = unique_datetime - timestmap_df_c2_previous.loc[timestmap_df_c2_previous.index[-1], 'Timestampms']
        #print(different_previous_c2)
        timestmap_df_c2 = df_c2.loc[df_c2['EC2_Timestamp'] == unique_datetime]
        
        
        timestamp_to_find = df_c2.loc[timestmap_df_c2.index[-1]-1, 'Timestampms']
        #print (timestamp_to_find, df.loc[timestamp_df.index[0]-1, 'Timestampms'])

        # Find difference from previous and next timestamp
        difference_previous,  difference_next = find_difference(timestamp_df, unique_datetime, df)
        
        if not(difference_previous == 0 and difference_next == 0) and ((difference_previous <= 1000 and difference_next <= 1000) or
                                                                       (difference_previous == 1000 and difference_next == 0) or
                                                                       (difference_previous == 0 and difference_next == 1000)):
    
            # Split equally e.g 1,2,2,3
            df = split_equally(timestamp_df, df)
        
        elif difference_previous <= 1000 and difference_next >=2000 and  len(timestamp_df.index) < 3:
            if df.loc[timestamp_df.index[0],'Record Number'] < df.loc[timestamp_df.index[-1],'Record Number']:
                #if len(timestmap_df_c2.index) < len(timestamp_df.index) and different_previous_c2 < 1000:
#                 if len(timestmap_df_c2_previous) != len(timestmap_df_c2_after):
#                     # Split equally e.g 1,2,2,3
#                     df = split_equally(timestamp_df, df)
#                 else:
#                     df = split_forward(timestamp_df, df)
                    
                if timestamp_df_next.index[0]-timestamp_df.index[-1] == 1 or len(timestmap_df_c2_after)>1 or len(timestmap_df_c2_previous) == len(timestmap_df_c2_after):
                    df = split_forward(timestamp_df, df)
                else:
                    # Split equally e.g 1,2,2,3
                    df = split_equally(timestamp_df, df)
            else:
                # Split equally e.g 1,2,2,3
                df = split_equally(timestamp_df, df)
        
        elif difference_previous <= 1000 and difference_next >=2000 and  len(timestamp_df.index) > 2:
            
            # split evenly then fill e.g 6.66, 7, 7, 7, 7, 9 
            df = split_evenly_fill(timestamp_df, df, difference_previous, difference_next)
            
        elif difference_previous >= 1000 and difference_next <=1000 and  len(timestamp_df.index) > 2:
            # fill then split evenly 
            df = fill_split_evenly(timestamp_df, df, difference_previous, difference_next)
                
        elif difference_previous == difference_next and difference_previous>1000 and difference_next>1000:
            if df.loc[timestamp_df.index[0],'Record Number'] < df.loc[timestamp_df.index[-1],'Record Number']:
                # fill preceding followed by behind
                df = split_forward_backward(timestamp_df, df)
            else:
                # Split equally e.g 1,2,2,3
                df = split_equally(timestamp_df, df)
                
        elif difference_previous > 1000 and difference_next <=2000 and  len(timestamp_df.index) < 3:
            if df.loc[timestamp_df.index[0],'Record Number'] < df.loc[timestamp_df.index[-1],'Record Number']:
                #if len(timestmap_df_c2.index) < len(timestamp_df.index):
#                 if len(timestmap_df_c2_previous) != len(timestmap_df_c2_after):
#                     # Split equally e.g 1,2,2,3
#                     df = split_equally(timestamp_df, df)
#                 else:
#                     # Fill Backward e.g 2,4,4,5
#                     df = split_backward(timestamp_df, df)
# 
                #len(timestmap_df_c2_previous)>1    or len(timestmap_df_c2_previous) != len(timestmap_df_c2_after)  len(timestmap_df_c2)>len(timestamp_df) or len(timestmap_df_c2)>1
#                 if (not timestamp_df_previous.empty and timestamp_df.index[0]-timestamp_df_previous.index[-1] == 1) \
#                     or (len(timestmap_df_c2)>1 and len(timestmap_df_c2_previous) != len(timestmap_df_c2_after))\
#                     or len(timestmap_df_c2_previous) == len(timestmap_df_c2_after) \
#                     or len(timestmap_df_c2) == len(timestamp_df):
#                     # Fill Backward e.g 2,4,4,5
#                     df = split_backward(timestamp_df, df)
#                 else:
#                     # Split equally e.g 1,2,2,3
#                     df = split_equally(timestamp_df, df)
                
                timestamp_df_previous_2 = df.loc[df['Timestamp'] == unique_datetime-2000]
                timestamp_df_c2_previous_2 = df_c2.loc[df_c2['EC2_Timestamp'] == unique_datetime-2000]
                
                    
                if len(timestamp_df) > len(timestmap_df_c2) \
                    and len(timestmap_df_c2_previous) > len(timestmap_df_c2_after) \
                    and len(timestamp_df_c2_previous_2) <= len(timestmap_df_c2_previous) \
                    and len(timestmap_df_c2_previous) > len(timestamp_df) \
                    and len(timestamp_df_c2_previous_2) > len(timestamp_df):
                    #and len(timestamp_df_c2_previous_2) > len(timestamp_df_previous_2):
                    # Split equally e.g 1,2,2,3
                    df = split_equally(timestamp_df, df)
                else:
                    # Fill Backward e.g 2,4,4,5
                    df = split_backward(timestamp_df, df)
                    
            else:
                # Split equally e.g 1,2,2,3
                df = split_equally(timestamp_df, df)
            
        
    # Add Prefix to Columns name (e.g. EC2/EOD_***) except Timestampms
    df.columns = df.columns.map(lambda x : 'EOD_'+x if x !='Timestampms' else x)
        
    # Shift Timestampms to first column
    df = df[ ['Timestampms'] + [ col for col in df.columns if col != 'Timestampms' ] ]
            
    return df

In [7]:
def process_dataframe(df, name):
    
    df =  clean_dataframe(df)
    
    # Find duplicates datetime in column 'Timestampms'
    df_duplicate = df[df.duplicated('Timestampms',keep=False)]
    previous_datetime = 0
    consec_flag = False
    for index, unique_datetime in enumerate (df_duplicate['Timestampms'].unique()):
        timestamp_df = df_duplicate.loc[df_duplicate['Timestampms'] == unique_datetime]
        print(unique_datetime)      
        # Find difference from previous and next timestamp
        difference_previous,  difference_next = find_difference(timestamp_df, unique_datetime, df)
        
        
        # In record order (asecending)
        if df.loc[timestamp_df.index[0],'Record Number'] < df.loc[timestamp_df.index[-1],'Record Number']:
            # Fill Forward e.g 1,2,2,4
            if difference_previous >= 1000 and difference_next >=2000 and (difference_previous<difference_next):
                df = split_forward(timestamp_df, df)
            # Fill Backward e.g 2,4,4,5
            elif difference_previous >= 2000 and difference_next == 1000:
                df = split_backward(timestamp_df, df)
            # Fill Forward Backward (Forward first) e.g 1,3,3,3,5
            elif(difference_previous == difference_next) and (difference_previous>1000 and difference_next>1000): 
                df = split_forward_backward(timestamp_df, df)
                
            elif not(difference_previous == 0 and difference_next == 0) and ((difference_previous == difference_next) or 
                                                                       (difference_previous <= 1000 and difference_next <= 1000) or
                                                                       (difference_previous == 1000 and difference_next == 0) or
                                                                       (difference_previous == 0 and difference_next == 1000)):
                # Split equally e.g 1,2,2,3
                df = split_equally(timestamp_df, df) 
        
        else:
            if not(difference_previous == 0 and difference_next == 0) and ((difference_previous == difference_next) or 
                                                                       (difference_previous <= 1000 and difference_next <= 1000) or
                                                                       (difference_previous == 1000 and difference_next == 0) or
                                                                       (difference_previous == 0 and difference_next == 1000)):
    
                # Split equally e.g 1,2,2,3
                df = split_equally(timestamp_df, df)
            
            else:
                # Split equally e.g 1,2,2,3
                df = split_equally(timestamp_df, df)
    
    
    # Add Prefix to Columns name (e.g. EC2/EOD_***) except Timestampms
    if name == 'C2':
        df.columns = df.columns.map(lambda x : 'EC2_'+x if x !='Timestampms' and x !='Timestamp' else x)
    elif name == 'OD':
        df.columns = df.columns.map(lambda x : 'EOD_'+x if x !='Timestampms' and x !='Timestamp' else x)
        
    # Shift Timestampms to first column
    df = df[ ['Timestampms'] + [ col for col in df.columns if col != 'Timestampms' ] ]
            
    return df

def process_timestamp(df):
    # Find duplicates datetime in colum 'Date'
    df_duplicate = df[df.duplicated('Timestampms',keep=False)]
    for unique_datetime in df_duplicate['Timestampms'].unique():
        timestamp_df = df_duplicate.loc[df_duplicate['Timestampms'] == unique_datetime]
        # Find difference from previous and next timestamp
        difference_previous,  difference_next = find_difference(timestamp_df, unique_datetime, df)
        # Check if both (index[0]-1) and (index[-1]+1) are only difference of 1000 millisecond apart respectively
        if not(difference_previous == 0 and difference_next == 0) and ((difference_previous == difference_next) or 
                                                                       (difference_previous <= 1000 and difference_next <= 1000) or
                                                                       (difference_previous == 1000 and difference_next == 0) or
                                                                       (difference_previous == 0 and difference_next == 1000)):
            # Split equally e.g 1,2,2,3
            df = split_equally_overall(timestamp_df, df)

    # Group resulting same real timestamp together
    df = df.groupby("Timestampms").last().reset_index()
    
    # Remove similar record numbers for both EC2 and EOD
    #df_duplicate = df[df.duplicated('EC2_Record Number',keep='first')]
    filter_col = [col for col in df if col.startswith('EC2_')]
    df.loc[df.duplicated('EC2_Record Number',keep='first'), filter_col] = np.NaN
    filter_col = [col for col in df if col.startswith('EOD_')]
    df.loc[df.duplicated('EOD_Record Number',keep='first'), filter_col] = np.NaN
        
    return df


In [56]:
def find_difference(timestamp_df, unique_datetime, df):
    difference_previous, difference_next = 0 , 0
    if timestamp_df.index[0]-1 >=0 and timestamp_df.index[-1]+1 <= len(df)-1:
        difference_previous = unique_datetime - df.loc[timestamp_df.index[0]-1,'Timestampms']
        difference_next = df.loc[timestamp_df.index[-1]+1,'Timestampms'] - unique_datetime
    elif timestamp_df.index[-1] == len(df)-1:
        difference_previous = unique_datetime - df.loc[timestamp_df.index[0]-1,'Timestampms']
    elif timestamp_df.index[0] == 0:
        difference_next = df.loc[timestamp_df.index[-1]+1,'Timestampms'] - unique_datetime
    
    return difference_previous, difference_next

def split_equally(timestamp_df, df):
    print("--- Handling Duplicates, Split duplicate evenly---")
    # For every Timestamp (1000 millisecond), find the interval
    interval_ms = round(1000 / len(timestamp_df.index))
    for counter in range (0, len(timestamp_df.index)):
        df.loc[timestamp_df.index[counter],'Timestampms'] = df.loc[timestamp_df.index[counter],'Timestampms'] + counter*interval_ms
        print(df.loc[timestamp_df.index[counter],'Record Number'], df.loc[timestamp_df.index[counter],'Date'],
              df.loc[timestamp_df.index[counter],'Timestampms'],df.loc[timestamp_df.index[counter],'Record Number'])
    return df

def split_equally_overall(timestamp_df, df):
    print("--- Handling Duplicates, Split duplicate evenly---")
    # For every Timestamp (1000 millisecond), find the interval
    interval_ms = round(1000 / len(timestamp_df.index))
    for counter in range (0, len(timestamp_df.index)):
        df.loc[timestamp_df.index[counter],'Timestampms'] = df.loc[timestamp_df.index[0],'Timestampms'] + counter*interval_ms
        print(df.loc[timestamp_df.index[counter],'Timestampms'],df.loc[timestamp_df.index[counter],'EOD_Record Number'])
    return df

def split_forward(timestamp_df, df):
    print("---Handling Duplicates, Fill gap with duplicate, Forward---")
    for counter in range (0, len(timestamp_df.index)):
        df.loc[timestamp_df.index[counter],'Timestampms'] = df.loc[timestamp_df.index[counter],'Timestampms'] + counter*1000
        print(df.loc[timestamp_df.index[counter],'Record Number'], df.loc[timestamp_df.index[counter],'Date'],
              df.loc[timestamp_df.index[counter],'Timestampms'],df.loc[timestamp_df.index[counter],'Record Number'])
    return df

def split_backward(timestamp_df, df):
    # Loop backwards
    print("---Handling Duplicates, Fill gap with duplicate, Backward---")
    count = 0
    for counter in range (len(timestamp_df.index), 0, -1):
        df.loc[timestamp_df.index[counter-1],'Timestampms'] = df.loc[timestamp_df.index[counter-1],'Timestampms'] - count*1000
        count +=1
        print(df.loc[timestamp_df.index[counter-1],'Record Number'], df.loc[timestamp_df.index[counter-1],'Date'],
              df.loc[timestamp_df.index[counter-1],'Timestampms'],df.loc[timestamp_df.index[counter-1],'Record Number'])
    
    return df

def split_forward_backward(timestamp_df, df):
    print("---Handling Duplicates, Fill gap with duplicate, Forward and Backward---")
    mid_point = len(timestamp_df.index) // 2
    left_count, right_count = 0, 0
    for counter in range (1, len(timestamp_df.index)):
        if counter % 2 == 0:
            right_count +=1
            df.loc[timestamp_df.index[mid_point+right_count],'Timestampms'] = df.loc[timestamp_df.index[mid_point],'Timestampms'] + right_count*1000
            print(df.loc[timestamp_df.index[mid_point+right_count],'Record Number'], df.loc[timestamp_df.index[mid_point+right_count],'Date'], df.loc[timestamp_df.index[mid_point+right_count],'Timestampms'])
        else:
            left_count +=1
            df.loc[timestamp_df.index[mid_point-left_count],'Timestampms'] = df.loc[timestamp_df.index[mid_point],'Timestampms'] - left_count*1000
            print(df.loc[timestamp_df.index[mid_point-left_count],'Record Number'], df.loc[timestamp_df.index[mid_point-left_count],'Date'], df.loc[timestamp_df.index[mid_point-left_count],'Timestampms'])
    return df

def split_evenly_fill(timestamp_df, df, difference_previous, difference_next):
    print("---Handling Duplicates, split evenly then fill forward---")
    difference_time_period = int((difference_next - difference_previous) // 1000)
    # Split evenly first
    interval_ms = round(1000 / (len(timestamp_df.index)-difference_time_period))
    for counter in range (0, (len(timestamp_df.index)-difference_time_period)):
        df.loc[timestamp_df.index[counter],'Timestampms'] = df.loc[timestamp_df.index[counter],'Timestampms'] + counter*interval_ms
        print(df.loc[timestamp_df.index[counter],'Record Number'], df.loc[timestamp_df.index[counter],'Date'],
              df.loc[timestamp_df.index[counter],'Timestampms'],df.loc[timestamp_df.index[counter],'Record Number'])
    # Fill in the gap for forward
    for counter in range ((len(timestamp_df.index)-difference_time_period), len(timestamp_df.index)):
        df.loc[timestamp_df.index[counter],'Timestampms'] = df.loc[timestamp_df.index[counter],'Timestampms'] + 1000
        print(df.loc[timestamp_df.index[counter],'Record Number'], df.loc[timestamp_df.index[counter],'Date'],
              df.loc[timestamp_df.index[counter],'Timestampms'],df.loc[timestamp_df.index[counter],'Record Number'])
        
    return df

def fill_split_evenly(timestamp_df, df, difference_previous, difference_next):
    print("---Handling Duplicates, fill then split evenly---")
    difference_time_period = int(difference_previous // difference_next)
    print(difference_time_period)
    # Fill in the gap for forward
    count = 1
    for counter in range (difference_time_period, 0, -1):
        df.loc[timestamp_df.index[counter-1],'Timestampms'] = df.loc[timestamp_df.index[counter-1],'Timestampms'] - count*1000
        count +=1
        print(df.loc[timestamp_df.index[counter-1],'Record Number'], df.loc[timestamp_df.index[counter-1],'Date'],
              df.loc[timestamp_df.index[counter-1],'Timestampms'],df.loc[timestamp_df.index[counter-1],'Record Number'])
    
    # Split evenly 
    interval_ms = round(1000 / (len(timestamp_df.index)-difference_time_period))
    for counter in range (difference_time_period,  len(timestamp_df.index)):
        df.loc[timestamp_df.index[counter],'Timestampms'] = df.loc[timestamp_df.index[counter],'Timestampms'] + (counter-difference_time_period)*interval_ms
        print(df.loc[timestamp_df.index[counter],'Record Number'], df.loc[timestamp_df.index[counter],'Date'],
              df.loc[timestamp_df.index[counter],'Timestampms'],df.loc[timestamp_df.index[counter],'Record Number'])
    
    return df

def output(sample_output_filename, df_result):
    # Import Output File
    df_output = pd.read_csv(sample_output_filename)
    
    df_combine = pd.DataFrame()
    df_combine = df_combine.assign(epoch = df_output['epoch'])
    df_combine = df_combine.assign(Timestampms = df_result['Timestampms']) 
    df_combine = df_combine.assign(EC2_Record_Number = df_result['EC2_Record Number']) 
    df_combine = df_combine.assign(EC2_Date = df_result['EC2_Date']) 
    df_combine = df_combine.assign(EOD_Record_Number = df_result['EOD_Record Number']) 
    df_combine = df_combine.assign(EOD_Date = df_result['EOD_Date']) 

    
    df_combine.to_csv('./result_combine_1.csv', index=False, header=True)
    
def unit_test(sample_output_filename, name, df_test):
    print("---Unit Test for " + name + " Dataframe---")
    # Unit Test for C2
    # Import Output File
    df_output = pd.read_csv(sample_output_filename)
    # Only retrieve respective columns
    if name == 'C2':
        df_output = df_output.drop(df_output[(df_output['EC2_002_Record_Number'].isnull())].index)
        #drop column with prefix EOD
        df_output = df_output.loc[:, ~df_output.columns.str.startswith('EOD')]
    elif name == 'OD':
        df_output = df_output.drop(df_output[(df_output['EOD_002_Record_Number'].isnull())].index)
        #drop column with prefix EOD
        df_output = df_output.loc[:, ~df_output.columns.str.startswith('EC2')]
    
    df_output['epoch'] = df_output.epoch.values.astype(np.float64)
    df_output = df_output.reset_index(drop=True)
    print(df_output.shape, df_test.shape)    
    #Output to CSV
    #df_output.to_csv('./df_output.csv', index=False, header=True)

    #Output to CSV
    df_test.to_csv('./df_test.csv', index=False, header=True)
    #df_drop_test = pd.read_csv('./df_test.csv')
    df_drop_test = df_test
    df_test['result'] = np.where(df_test['Timestampms'] == df_output['epoch'], 'True', 'False')
    df_test.to_csv('./df_test.csv', index=False, header=True)
    #df_drop_test = df_drop_test.sort_values(by='ATO_Real_Timestampms',ascending=True).reset_index(drop=True)
    # Assert whether sample output and self processed are equal
    assert_equal = omap.nan_equal(df_drop_test['Timestampms'].values, df_output['epoch'].values)
    print("Equality Between Sample Output and Self Processed: ", assert_equal)
    assert_equal = omap.nan_equal(df_drop_test['E'+name+'_Record Number'].values, df_output['E'+name+'_002_Record_Number'].values)
    print("Equality Between Sample Output and Self Processed: ", assert_equal)
    
    #df_drop_test.columns = df_output.columns
    #print(np.testing.assert_allclose(df_drop_test.values, df_output.values, rtol=1e-10, atol=0))
    #print(pd.testing.assert_frame_equal(df_drop_test, df_output, check_dtype=False))
    #print(df_drop_test.compare(df_output, align_axis=0))
    #assert_equal = nan_equal(df_drop_test.values, df_output.values)
    #assert_equal = nan_equal(df_drop['ATO_* General'].values, df_output['ATO_0101__General'].values)
    #print("Equality Between Sample Output and Self Processed: ", assert_equal)
    #print(np.testing.assert_equal(df_drop_test.values, df_output.values))
    
def unit_test_all(sample_output_filename, df_test):
    print("---Unit Test for entire Dataframe---")
    # Import Output File
    df_output = pd.read_csv(sample_output_filename)
    print(df_output.shape, df_test.shape)
    assert_equal = omap.nan_equal(df_test['Timestampms'].values, df_output['epoch'].values)
    print("Equality Between Sample Output and Self Processed: ", assert_equal)
    #df_test['result'] = np.where(df_test['Timestampms'] == df_output['epoch'], 'True', 'False')
    #df_test.to_csv('./df_test.csv', index=False, header=True)
        
    #df_test['result_1'] = np.where(df_test['EC2_Record Number'] == df_output['EC2_002_Record_Number'], 'True', 'False')
    #df_test.to_csv('./df_test.csv', index=False, header=True)
    
    assert_equal = omap.nan_equal(df_test['EC2_Record Number'].values, df_output['EC2_002_Record_Number'].values)
    print("Equality Between Sample Output and Self Processed: ", assert_equal)
    
    assert_equal = omap.nan_equal(df_test['EOD_Record Number'].values, df_output['EOD_002_Record_Number'].values)
    print("Equality Between Sample Output and Self Processed: ", assert_equal)
    
    print(np.testing.assert_equal(df_test['EC2_Record Number'].values, df_output['EC2_002_Record_Number'].values))
    print(np.testing.assert_equal(df_test['EOD_Record Number'].values, df_output['EOD_002_Record_Number'].values))
    
    df_result_2 = df_test.copy(deep=True)
    df_result_2.columns = df_output.columns
    print(pd.testing.assert_frame_equal(df_result_2, df_output, check_dtype=False, check_column_type=False))
    df_result_2.compare(df_output, align_axis=0)